In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm_notebook
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test = pd.read_csv("../input/conways-reverse-game-of-life-2020/test.csv", index_col=0)
test.head()

# Observe the train data

In [ ]:
trainData = pd.read_csv("../input/conways-reverse-game-of-life-2020/train.csv", index_col=0)
trainData.head()

In [ ]:
trainData.info()

In [ ]:
trainData.duplicated().sum()

In [ ]:
print("Number before dropping : %d " % trainData.shape[0])
trainData = trainData.drop(index=trainData[trainData.duplicated()].index)
print("Number after dropping : %d " % trainData.shape[0])

In [ ]:
trainData.head()

### Review Rules
1. Overpopulation: if a living cell is surrounded by more than three living cells, it dies.
2. Stasis: if a living cell is surrounded by two or three living cells, it survives.
3. Underpopulation: if a living cell is surrounded by fewer than two living cells, it dies.
4. Reproduction: if a dead cell is surrounded by exactly three cells, it becomes a live cell.

In [ ]:
N_ROWS = 25
N_COLS = 25
def getFlowBoards(index):
    start = trainData.iloc[index][1:626].to_numpy()
    startBoard = start.reshape(25, 25)
    boards = {0: startBoard}
    tmp = np.array(startBoard, copy=True)
    delta = trainData.iloc[index][0]
    for i in range(delta):
        for row in range(N_ROWS):
            for col in range(N_COLS):
                current = boards[i][row][col]
                arround = boards[i].take(range(row-1,row+2),mode='wrap', axis=0).take(range(col-1,col+2),mode='wrap',axis=1)
                arroundSum = arround.sum()
                arroundSum = arroundSum - current
                if current == 0 and arroundSum == 3:
                    tmp[row][col] = 1
                elif current == 1:
                    if arroundSum > 3 or arroundSum < 2:
                        tmp[row][col] = 0
                    else:
                        tmp[row][col] = 1
        boards[i + 1] = np.array(tmp, copy=True)
    return boards;

def plotBoards(boards):
    colors = ['white','black'] 
    cmap = mpl.colors.ListedColormap(colors)
    plt.rcParams['figure.figsize'] = (15.0, 15.0) 
    plt.figure()
    size = len(boards)
    for key, value in boards.items():
        plt.subplot(1,size,key + 1)
        plt.xlabel(key)
        plt.imshow(value ,cmap = cmap)

In [ ]:
plotBoards(getFlowBoards(0))

In [ ]:
plotBoards(getFlowBoards(1))

In [ ]:
plotBoards(getFlowBoards(2))

In [ ]:
plotBoards(getFlowBoards(3))

In [ ]:
plotBoards(getFlowBoards(4))

# Valid and Generate More Boards

In [ ]:
def getMovedBoard(start, delta):
    board = np.array(start, copy=True)
    tmp = np.array(start, copy=True)
    for i in range(delta):
        for row in range(N_ROWS):
            for col in range(N_COLS):
                current = board[row][col]
                arround = board.take(range(row-1,row+2),mode='wrap', axis=0).take(range(col-1,col+2),mode='wrap',axis=1)
                arroundSum = arround.sum()
                arroundSum = arroundSum - current
                if current == 0 and arroundSum == 3:
                    tmp[row][col] = 1
                elif current == 1:
                    if arroundSum > 3 or arroundSum < 2:
                        tmp[row][col] = 0
                    else:
                        tmp[row][col] = 1
        board = np.array(tmp, copy=True)
    return board;

In [ ]:
total = trainData.shape[0]
# validBoards = {}
delta1TrainSource = []
delta1TrainTarget = []
delta2TrainSource = []
delta2TrainTarget = []
delta3TrainSource = []
delta3TrainTarget = []
delta4TrainSource = []
delta4TrainTarget = []
delta5TrainSource = []
delta5TrainTarget = []

for i in tqdm_notebook(range(total)):
    delta = trainData.iloc[i][0]
    
    startData = trainData.iloc[i][1:626].to_numpy()
    start = startData.reshape(25, 25)
    
    stopData = trainData.iloc[i][626:].to_numpy()
    stop = stopData.reshape(25, 25)
    
#     if (np.array_equal(getMovedBoard(start, delta), stop)):
#         validBoards[i] = start
    if (delta == 1):
        delta1TrainSource.append(stop)
        delta1TrainTarget.append(start)
    if (delta == 2):
        delta2TrainSource.append(stop)
        delta2TrainTarget.append(start)
    if (delta == 3):
        delta3TrainSource.append(stop)
        delta3TrainTarget.append(start)
    if (delta == 4):
        delta4TrainSource.append(stop)
        delta4TrainTarget.append(start)
    if (delta == 5):
        delta5TrainSource.append(stop)
        delta5TrainTarget.append(start)
            
print("number of delta 1 data = {0}".format(len(delta1TrainSource)))
print("number of delta 2 data = {0}".format(len(delta2TrainSource)))
print("number of delta 3 data = {0}".format(len(delta3TrainSource)))
print("number of delta 4 data = {0}".format(len(delta4TrainSource)))
print("number of delta 5 data = {0}".format(len(delta5TrainSource)))

# Save to CSV

In [ ]:
trainSource = np.array([delta1TrainSource, delta2TrainSource, delta3TrainSource, delta4TrainSource, delta5TrainSource])
trainTarget = np.array([delta1TrainTarget, delta2TrainTarget, delta3TrainTarget, delta4TrainTarget, delta5TrainTarget])

In [ ]:
for i in tqdm_notebook(range(5)):
    pd.DataFrame(data=np.array(trainSource[i]).reshape(-1, 625), columns=['start_' + str(_) for _ in range(0, 625)]).to_csv("train_delta{}_source.csv".format(i+1))
    pd.DataFrame(data=np.array(trainTarget[i]).reshape(-1, 625), columns=['start_' + str(_) for _ in range(0, 625)]).to_csv("train_delta{}_target.csv".format(i+1))